In [1]:
import numpy as np
import random
import statsmodels.api as sm
import os
from statsmodels.sandbox.stats.multicomp import multipletests
import pybedtools
from pyBedGraph import BedGraph
from pybedtools import BedTool
from sys import argv
import matplotlib.pyplot as plt

import time
import tracemalloc

In [3]:
# FDR: 0.1
# sample_size = 5000
# For 'chr1':
#   # Pass
#   # Fail
#   # Runtime/memory

# Read in samples from bed/bedgraph file
def read_gems(directory, file_name, chr_name):
    """
    Read a GEM file of the form "PlinePgem".
    Args:
       directory (str): directory of the file location (ex: '/Users/kimm/')
       file_name (str): name of the file (ex: 'SHG0008H.Fragnum_PlinePgem')
    Returns:
       in_gems (list): tab-separated lists of lists
    """
    subset_gems = []
    cnt_all = 0
    cnt_subset = 0
    with open(directory + file_name) as f:
        for line in f:
            tmp = line.strip().split("\t")
            if tmp[0] == chr_name:
                subset_gems.append(tmp)
                cnt_subset += 1
            cnt_all += 1
        #in_gems = [line.strip().split("\t") for line in f]
    gem_span = int(subset_gems[0][2]) - int(subset_gems[0][1])
    return subset_gems, cnt_all, cnt_subset, gem_span

7508508
648188


In [4]:
# Read the hg38.chrom.sizes file into a dictionary
def read_chroms(directory, genome_name):
    """
    Read a tab-delimited text file with list of chromosomes and their sizes.
    Args:
       directory (str): directory of the file location (ex: '/Users/kimm/')
       genome_name (str): name of reference genome (ex: 'dm3', 'mm10', 'hg38', 'hg19', etc.)
                        Note: must have a text file <genome_name>.chrom.sizes in the directory.
    Returns:
       chrom_dict (dictionary): tab-separated lists of lists
    """
    chrom_dict = {}
    with open(directory + genome_name + '.chrom.sizes') as f:
        for line in f:
            tmp_list = line.strip().split("\t")
            chrom_dict[tmp_list[0]] = int(tmp_list[1])
    return chrom_dict


# Determine random start locations for samples to compare to observed sample
# Gem span should be the same for all single-end fragments
def random_loc(chrom_size, gem_span, chrom, sample_size, cov):
    """
    Randomly locate GEM with same span in a chromosome sample_size times.
    Args:
       chrom_size (int): size of a given chromosome (ex: 23011544 if 'chr2L')
       gem_span (int): GEM from start of first fragment to end of last fragment (ex: 5000)
       sample_size (int): number of random locations to sample
    Returns:
       startpos (array): random integers array of length 'sample_size'
    """
    np.random.seed(12345)
    startpos_lst = []
    pseudo = []
    exp_enrich_lst = []
    i = 0
    while i < sample_size:
        startpos = np.random.randint(low = 0, high = chrom_size - gem_span - 1)
        current_frag = [chrom, startpos, startpos + gem_span]
        exp_enrich = cov.stats(stat = 'max', intervals = [current_frag])[0]
        #outfile.write("CurrentIndex: " + str(i) + "\t")
        #outfile.write("Exp Enrichment: " + str(exp_enrich) + "\n")
        #outfile.write("Current Pseudo: " + str(pseudo) + "\n")
        if int(exp_enrich) == -1:
            i -= 1
        else:
            startpos_lst.append(startpos)
            pseudo.append(current_frag)
            exp_enrich_lst.append(exp_enrich)
        #outfile.write("CurrentIndex SameCycle: " + str(i) + "\t")
        #outfile.write("Exp Enrichment SameCycle: " + str(exp_enrich) + "\n")
        #outfile.write("Current Pseudo SameCycle: " + str(pseudo) + "\n")
        i += 1
    return(startpos_lst, pseudo, exp_enrich_lst)

#def get_raw_pval(obs_frags, obs_span, obs_fraglen, obs_f2f, sample_size, cov, bin_size, chrom_size):
def get_raw_pval(exp_enrich, obs_frags, sample_size, cov):
    """
    Compute raw p-value for a GEM.
    Args:
       obs_frags (list of list): [chrom,start,end] for each fragment 
       obs_span (int): start of leftmost fragment to end of rightmost fragment
       obs_fraglen (list): length of each fragment
       obs_f2f (list): distance between neighboring fragments
       sample_size (int): number of pseudo-GEMs to sample
       cov: bedgraph coverage track
       bin_size (int): size of the bin used to generate cov
       chrom_size (int): size of the chromosome
    Returns:
       raw_pval (float): raw p-value computed
       obs_enrich (int): observed enrichment
    """
    # observed enrichment
#    frag_cov = []
#    for i in range(len(obs_frags)):
#        frag_cov.append(get_mean_cov(obs_frags[i][0], obs_frags[i][1], cov, bin_size))
    #frag_cov = list(cov.stats(stat = 'mean', intervals = [[obs_frags[0], int(obs_frags[1]), int(obs_frags[2])]]))
    obs_enrich = cov.stats(stat = 'max', intervals = [[obs_frags[0], int(obs_frags[1]), int(obs_frags[2])]])[0]
    #obs_enrich = sum(frag_cov)/len(frag_cov)
    #obs_enrich = max(frag_cov)    
    # expected enrichment (sampling background)
    #startpos = random_loc(chrom_size, obs_span, sample_size)
    '''
    exp_enrich = []
    for k in range(sample_size):
        pseudo = pseudo_gem(startpos[k], obs_span, chr_name)
        exp = list(cov.stats(stat = 'mean', intervals = pseudo))
        exp_enrich.append(sum(exp)/len(exp))
        #exp_enrich.append(max(exp))
    '''
    # write 1000 null (expected, sampled) values; 20191226
#    prefix_null = "_".join(prefix.split("_")[:-1]) + "_1000"
#    with open(out_directory + prefix_null + "_enrichTest_null.txt", "a") as f1:
#        f1.write(','.join(map(str, [round(x,1) for x in exp_enrich])) + '\n')
#    f1.close()
    # compute raw p-value
    #raw_pval = sum(i > obs_enrich for i in exp_enrich)/sample_size
    tot = 0
    for i in exp_enrich:
        tot += (i >= obs_enrich)
    raw_pval = tot/sample_size
    return(raw_pval, obs_enrich)

def get_adj_pval(raw_pval_list, fdr_thresh, method):
    """ 
    Adjust raw pvalues by Benjamini Hochberg multiple testing adjustment. 
    Args:
       raw_pval_list (list): list of raw p-values (ex: [0.1, 0.04, 0.1])
       fdr_thresh (float): false discovery rate (ex: 0.05)
       method (string): adjustment method (ex: 'fdr_bh')
    Returns:
       adj_pval_list (array): array of booleans and adjusted p-values (ex: [0.1, 0.1, 0.1])
    """
    adj_pval_list = multipletests(raw_pval_list, alpha = fdr_thresh, method = method)
    return(adj_pval_list)

def write_master_result(out_gem_list, out_name):
    """ 
    Write out significance test results.
    Args: 
       out_gem_list (list): list with 11 items including gem_id, frag_str, p-values, etc.
       out_name (string): output file name
    Returns:
       None
    """
    with open(out_name, 'a') as file1:
        header = ['GEM_ID', 'Start Coord', 'End Coord', 'Category', 'Obs', 'rawpval1', 'adjpval1', 'decis1', 'rawpval2', 'adjpval2', 'decis2']
        file1.write('\t'.join(map(str, header)) + '\n')
        for i in range(len(out_gem_list)):
            file1.write('\t'.join(map(str, out_gem_list[i])) + '\n')

def write_output_beds(lst, out_name):
    with open(out_name, 'a') as file:
        for line in lst:
            file.write('\t'.join(line) + '\n')

In [5]:
if __name__ == '__main__':
    #tracemalloc.start()
    total_start = time.time()
    argv = os.environ.get('NB_ARGS')
    argv = argv.split(" ")
    ### Set parameters ###
    library_name = argv[0] ## Library name of our data ##
    genome_name = argv[1] ## Name of the reference genome ##
    fdr_thresh = float(argv[2])  # should be argument; Benjamini-Hochberg FDR; p-value cutoff ##
    chr_lst = argv[3] # should be argument
    samp_size = int(argv[4]) ## Number of pseudo-GEMs ##
#    bin_size = int(argv[6])
    bg_name = argv[5] # bedgraph file name
    directory = argv[6]
    file_name = argv[7]
    strand_type = argv[8] ## Paired End or Single End (PE or SE) ##
    out_directory = directory + library_name + "_EnrichTest_FDR_" + str(fdr_thresh) + '/'
    if not os.path.exists(out_directory):
        os.mkdir(out_directory)

    chr_lst = chr_lst.split("\n")
    print(str(chr_lst))
    for chr_name in chr_lst:
        prefix = library_name + "_" + chr_name + "_FDR_" + str(fdr_thresh) + "_pseudoGEM_" + str(samp_size) + str(strand_type)
    
        ### Set directory and input file name ###
        #gem_span = int(argv[8])
    
        #### Log file ####
        out = open(out_directory + prefix + "_enrichTest_logFile.txt", "a")
    
        #out.write("Software version: v1.0 (2020-04-01, Kim)" + "\n")
        out.write("Input directory: " + directory + "\n")
        out.write("Input file name: " + file_name + "\n")
        out.write("Input bedgraph file: " + bg_name + "\n")
        out.write("Output directory: " + out_directory + "\n")
        out.write("Library name: " + library_name + "\n")
        out.write("Reference genome: " + genome_name + "\n")
        out.write("FDR threshold: " + str(fdr_thresh) + "\n")
        out.write("Number of pseudo-GEMs: " + str(samp_size) + "\n")
        out.write("Single or Paired End: " + str(strand_type) + "\n")
        out.write("Started processing frequency-based enrichment test. \n")
        out.write("================================= \n")
        out.write("===== Chromosome is: " + chr_name + ". ===== \n")
        out.write("================================= \n")
    
        read_bed_start = time.time()
        subset_gems, cnt_all, cnt_subset, gem_span = read_gems(directory, file_name, chr_name)
        out.write("Finished reading the input GEM file. \n")
        out.write(str(cnt_all-1) + " total GEMs. \n")
        out.write("================================= \n")
        read_bed_end = time.time()
        
        ### Read chrom sizes file ###
        read_sizes_start = time.time()
        chrom_size = read_chroms(directory, genome_name)
        read_sizes_end = time.time()
        
        ### Subset input GEM files by chromosome ###
        #subset_gems = [x for x in in_gems if chr_name == x[1].split(":")[0]]
        #out.write("Finished subsetting GEMs. \n")
        out.write(str(cnt_subset) + " GEMs in " + chr_name + " (of " + str(cnt_all-1) + " total GEMs). \n")
        out.write("================================= \n")
        
        #del in_gems
        
        ### Read bedgraph coverage file ###
        #bg_name = library_name + '_binned_' + str(bin_size) + 'bp_' + chr_name + '.bedgraph'
        if not os.path.isfile(directory+bg_name):
            out.write("Error: no bedgraph coverage file." + "\n")
        #cov = BedGraph(directory + genome_name+".chrom.sizes", directory + bg_name, str(chr_name))
        pyBedGraph_start = time.time()
        cov = BedGraph(directory + genome_name+".chrom.sizes", directory + bg_name, [chr_name])
        cov.load_chrom_data(chr_name)
        #coverage = read_bedgraph(directory, library_name, bin_size, chr_name)
        out.write("Finished reading the coverage file " + bg_name + ". \n")
        out.write("================================= \n")
        pyBedGraph_end = time.time()
    
        ### Initialize output GEM list ###
        out_gems = [] 
        
        ### Calculate raw p-values for subset_gems ###
        pvals = []
    
        random_loc_start = time.time()
        start_pos, pseudo, exp_enrich = random_loc(chrom_size[chr_name], gem_span, chr_name, samp_size, cov)
        random_loc_end = time.time()
        #exp_enrich = []
        #pseudo = []
        
        #out.write("Pseudo: " + str(pseudo[i]) + "\n")
        #out.write("Exp_enrich: " + str(exp_enrich) + "\n")
            
        #exp_enrich.append(sum(exp)/len(exp))
        out.write("Finished generating pseudo samples.\n")
        out.write("================================= \n")
        raw_pvals_start = time.time()
        for k in range(len(subset_gems)):
            gem_id = subset_gems[k][3]
            #gem_id, chrom, frags, span, fraglen, f2fdist, fragnum, frag_str, coord = list(extract_info(subset_gems[k]))
            pval, enr = get_raw_pval(exp_enrich, subset_gems[k], samp_size, cov)
            pvals.append(pval)
            #out.write("Pval: " + str(pval) + "\n")
            out_gems.append([gem_id, subset_gems[k][1], subset_gems[k][2], 'Orig', round(enr,1), round(pval,3)])
        out.write("Finished calculating raw p-values for " + str(k + 1) + " GEMs. \n")
        #raw_pass = sum(i <= fdr_thresh for i in pvals)
        raw_pass = 0
        for i in pvals:
            raw_pass += (i <= fdr_thresh)
        out.write(str(raw_pass) + " GEMs have raw p-val <= " + str(fdr_thresh) + "(i.e., " + "{0:.2f}".format(raw_pass/(k+1)*100) +" %). \n")
        out.write("================================= \n")
        raw_pvals_end = time.time()
        
        ### Adjust p-values ###
        adj_pvals_start = time.time()
        pass_pileup = []
        fail_pileup = []
        adj_pval = get_adj_pval(pvals, fdr_thresh, 'fdr_bh')
        out.write("Finished adjusting p-values for " + str(k + 1) + " GEMs. \n")
        adj_pass = 0
        for i in range(len(adj_pval[1])):
            #print(adj_pval[0][i])
            out_gems[i].append(round(adj_pval[1][i], 3))
            if adj_pval[0][i]:
                out_gems[i].extend(['PASS','.','.','.'])
                pass_pileup.append(subset_gems[i])
                adj_pass += 1
            else:
                out_gems[i].extend(['FAIL','.','.','.'])
                fail_pileup.append(subset_gems[i])
        out.write("Pass Pileup: " + str(len(pass_pileup)) + "\n")
        out.write("Fail Pileup: " + str(len(fail_pileup)) + "\n")
        out.write(str(adj_pass) + " GEMs have adjusted p-val <= " + str(fdr_thresh) + "(i.e.," + "{0:.2f}".format(adj_pass/(k+1)*100) +" %). \n")
        out.write("================================= \n")
        adj_pvals_end = time.time()
        
        ### Write results ###
        write_res_start = time.time()
        write_master_result(out_gems, out_directory+prefix+'_enrichTest_master.txt')
        write_output_beds(pass_pileup, out_directory+prefix+'_pass_pileup.bed')
        write_output_beds(fail_pileup, out_directory+prefix+'_fail_pileup.bed')
        write_res_end = time.time()
        out.write("Finished writing files. \n")
        out.write("DONE. \n")
        total_end = time.time()
        out.write("================================= \n")
        out.write("Timings: \n")
        out.write("Reading Bed File: " + str(read_bed_end - read_bed_start) + "\n")
        out.write("Reading Sizes: " + str(read_sizes_end - read_sizes_start) + "\n")
        out.write("pyBedGraph: " + str(pyBedGraph_end - pyBedGraph_start) + "\n")
        out.write("Random Locations: " + str(random_loc_end - random_loc_start) + "\n")
        out.write("Raw Pvals: " + str(raw_pvals_end - raw_pvals_start) + "\n")
        out.write("Adjusted Pvals: " + str(adj_pvals_end - adj_pvals_start) + "\n")
        out.write("Writing Results: " + str(write_res_end - write_res_start) + "\n")
        out.write("Total: " + str(total_end - total_start) + "\n")
        out.write("================================= \n")
        #out.write("Memory Usage: \n")
        #out.write(str(tracemalloc.get_traced_memory()))
        #tracemalloc.stop()
        out.close()

AttributeError: 'NoneType' object has no attribute 'split'

5


2
